In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset
import gymnasium as gym

from astropy.io import fits
from datetime import datetime, timezone
import pandas as pd
import json
import fitsio
import time
import pickle
import re

%reload_ext autoreload
%autoreload 2

In [2]:
import survey_ops
from survey_ops.utils import units, geometry, interpolate
from survey_ops.coreRL.offline_dataset import OfflineDECamDataset
from survey_ops.coreRL.agents import Agent
from survey_ops.algorithms import DDQN, BehaviorCloning
from survey_ops.utils.sys_utils import seed_everything
from survey_ops.coreRL.data_processing import load_raw_data_to_dataframe


In [3]:
from survey_ops.utils import ephemerides
from tqdm import tqdm
from pathlib import Path

In [4]:
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from collections import Counter
from scipy.stats import entropy

In [5]:
SEED = 10
seed_everything(SEED)
torch.set_default_dtype(torch.float32)
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "cpu"   
)

# Load config and lookup files

In [6]:
with open('../configs/global_config.json', 'r') as f:
    gcfg = json.load(f)

In [7]:
with open('../experiment_results/az_grid_config.json', 'r') as f:
    cfg = json.load(f)

In [8]:
# with open(glob_cfg.get('paths.lookup_dir') + '/' + cfg.get('paths')['FIELD2NVISITS'], 'r') as f:
#     field2nvisits = json.load(f)
# with open(glob_cfg.get('paths.lookup_dir') + '/' + cfg.get('paths')['FIELD2NAME'], 'r') as f:
#     field2name = json.load(f)
# with open(glob_cfg.get('paths.lookup_dir') + '/' + cfg.get('paths')['FIELD2RADEC'], 'r') as f:
#     field2radec = json.load(f)
with open('../data/lookups/nside16_bin2azel.json', 'r') as f:
    bin2azel = json.load(f)


In [9]:
# fits_path = Path(cfg.get('paths.DFITS')).resolve().parents[1] / 'data' / cfg.get('paths.DFITS')
# json_path = Path(cfg.get('paths.DJSON')).resolve().parents[1] / 'data' / cfg.get('paths.DJSON')

df = load_raw_data_to_dataframe(fits_path=None, json_path='../data/fits/decam-exposures-20251211.json')

cfg['data']['specific_years'] = None
cfg['data']['specific_months'] = None
cfg['data']['specific_days'] = None
cfg['data']['include_bin_features'] = True

In [10]:
dat_fn = './full_azel_offline_dataset.pkl'

In [11]:
try:
    with open(dat_fn, 'rb') as f:
        d = pickle.load(f)
except:
    d = OfflineDECamDataset(
        df=df,
        cfg=cfg,
        gcfg=gcfg,
        )
    with open(dat_fn, 'wb') as f:
        pickle.dump(d, f)

In [12]:
states_np = d.states.detach().numpy()
actions_np = d.actions.detach().numpy()

# Behavior cloning theoretical accuracy limit

Bayes error rate, $P^*$ is the lowest possible error rate for any classifier of a random outcome

One way of estimating the error rate from the data is using a K neighbors classifier and calculating the expected prediction error as

$$
\text{BE} = P^* = E_x [ \sum_{k=1} ^ K L ( C_k, \hat{C} ( x ) ) P ( C_k | x ) ] 
$$

- $x$ is the data instance
- $L(C_k, \hat{C}(x))$ is the loss function, 0 if $x = y$ and 1 if they do not match 
- $P(C_k|x)$ is the conditional probability of label $k$ for instance $x$

Estimating Bayes error rate, $P^*$, using a k-NN classifier ($P_{kNN}$). Cover and Hart 1967 showed that the 1-NN classifier approaches $2P^*$ as number of samples grows to $\infty$ 

In [13]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import LeaveOneOut, cross_val_score

def estimate_bayes_error(states, actions):
    # Use 1-Nearest Neighbor
    knn = KNeighborsClassifier(n_neighbors=1)
    
    # Use Leave-One-Out Cross-Validation for the most stable estimate 
    # on smaller datasets, or standard K-Fold for larger ones.
    loo = LeaveOneOut()
    scores = cross_val_score(knn, states, actions, cv=loo)
    
    # The error rate of 1-NN
    p_knn = 1 - np.mean(scores)
    
    # According to Cover-Hart: P* <= P_knn <= 2P*(1-P*)
    # A common heuristic for the lower bound:
    lower_bound = 0.5 * p_knn
    upper_bound = p_knn
    
    return lower_bound, upper_bound

# Example Usage:
# lb, ub = estimate_bayes_error(your_features, your_labels)
# print(f"Estimated Bayes Error is between {lb:.4f} and {ub:.4f}")

In [15]:
lb, ub = estimate_bayes_error(states=states_np, actions=actions_np)


In [53]:
# nsamples = np.arange(10, len(rd_states_np), step=len(rd_states_np)//10)
# lubounds = np.empty(shape=(len(nsamples), 2))
# for i, _nsamp in enumerate(nsamples):
#     lb, ub = estimate_bayes_error(states=rd_states_np[:(i+1)*_nsamp], actions=rd_actions_np[:(i+1)*_nsamp])
#     lubounds[i] = np.array([lb, ub])